<a href="https://colab.research.google.com/github/Rojan116/NepaliHandWrittenAlphabetRecognizer/blob/master/new_cnn_may15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Rojan116/NepaliHandWrittenAlphabetRecognizer.git

Cloning into 'NepaliHandWrittenAlphabetRecognizer'...
remote: Enumerating objects: 92082, done.
remote: Counting objects: 100% (92082/92082), done.
remote: Compressing objects: 100% (92082/92082), done.
remote: Total 92082 (delta 3), reused 92073 (delta 0), pack-reused 0
Receiving objects: 100% (92082/92082), 56.75 MiB | 33.99 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (92004/92004), done.


In [2]:
ls

NepaliHandWrittenAlphabetRecognizer/  sample_data/


In [3]:
cd NepaliHandWrittenAlphabetRecognizer

/content/NepaliHandWrittenAlphabetRecognizer


In [10]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tqdm import tqdm
import random
from keras.utils import to_categorical


#%matplotlib.inline 

PATH = os.getcwd()
DATADIR = PATH + '/dhrcdata/Train'		
train_data_path = PATH + '/dhrcdata/Train'
test_data_path = PATH + '/dhrcdata/Test'


CATEGORIES = [

"character_1_ka","character_2_kha","character_3_ga","character_4_gha","character_5_kna",
"character_6_cha","character_7_chha","character_8_ja","character_9_jha","character_10_yna",
"character_11_taamatar","character_12_thaa","character_13_daa","character_14_dhaa","character_15_adna",
"character_16_tabala","character_17_tha","character_18_da","character_19_dha","character_20_na","character_21_pa",
"character_22_pha","character_23_ba","character_24_bha","character_25_ma","character_26_yaw","character_27_ra",
"character_28_la","character_29_waw","character_30_motosaw","character_31_petchiryakha","character_32_patalosaw",
"character_33_ha","character_34_chhya","character_35_tra","character_36_gya","digit_0","digit_1","digit_2","digit_3",
"digit_4","digit_5","digit_6","digit_7","digit_8","digit_9"

]

IMG_SIZE = 32
training_data = []
testing_data = []

def create_training_data():
  for catagory in CATEGORIES:
    path = os.path.join(train_data_path, catagory)
    class_num = CATEGORIES.index(catagory)
    for img in os.listdir(path):
        try:
          img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
          new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
          training_data.append([new_array,class_num])
        except Exception as e:
          pass



def create_testing_data():
  for catagory in CATEGORIES:
    path = os.path.join(test_data_path, catagory)
    class_num = CATEGORIES.index(catagory)
    for img in os.listdir(path):
        try:
          img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
          new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
          testing_data.append([new_array,class_num])
        except Exception as e:
          pass



create_training_data()
create_testing_data()

#shuffle training data for trainning diff image set in every batch
random.shuffle(training_data)

#hot encode image dataset

train_x = []
train_y = []
test_x = []
test_y = []



#seperating features and labels from list
for features, labels in training_data:
	train_x.append(features)
	train_y.append(labels)

for features, labels in testing_data:
	test_x.append(features)
	test_y.append(labels)


#converting list to array
train_y_array = np.array(train_y)
test_y_array = np.array(test_y)

#converting image dataset to input 32*32 dataset 

train_image_array = np.array(train_x).reshape(-1,32,32,1)
test_image_array = np.array(test_x).reshape(-1,32,32,1)


#one hot enoded class 
train_y = to_categorical(train_y_array)
test_y = to_categorical(test_y_array)


train_y = np.array(train_y)
test_y = np.array(test_y)

#all train x, y and test x y are in array form now


#now creating our model

from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

model = Sequential()

model.add(InputLayer(input_shape=(32,32,1)))
model.add(Conv2D(filters=32,kernel_size=3,strides=1,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Conv2D(filters=50,kernel_size=3,strides=1,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Conv2D(filters=80,kernel_size=3,strides=1,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))




model.add(Dense(46,activation='softmax'))
optimizer = Adam(lr=1e-3)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=train_image_array,y=train_y,epochs=30,batch_size=100, validation_split=0.3)

model.summary()



Train on 54740 samples, validate on 23460 samples
Epoch 1/30
54740/54740 [==============================] - 4s 74us/step - loss: 3.1583 - acc: 0.3715 - val_loss: 0.8026 - val_acc: 0.7541
Epoch 2/30
54740/54740 [==============================] - 3s 60us/step - loss: 0.8692 - acc: 0.7283 - val_loss: 0.4600 - val_acc: 0.8556
Epoch 3/30
54740/54740 [==============================] - 3s 59us/step - loss: 0.6045 - acc: 0.8120 - val_loss: 0.3692 - val_acc: 0.8850
Epoch 4/30
54740/54740 [==============================] - 3s 60us/step - loss: 0.4959 - acc: 0.8457 - val_loss: 0.3066 - val_acc: 0.9050
Epoch 5/30
54740/54740 [==============================] - 3s 60us/step - loss: 0.4279 - acc: 0.8677 - val_loss: 0.2668 - val_acc: 0.9182
Epoch 6/30
54740/54740 [==============================] - 3s 60us/step - loss: 0.3819 - acc: 0.8798 - val_loss: 0.2385 - val_acc: 0.9263
Epoch 7/30
54740/54740 [==============================] - 3s 60us/step - loss: 0.3472 - acc: 0.8914 - val_loss: 0.2789 - val_acc

In [0]:
model.save('androidtest_model.h5')
model.save_weights('androidtest_weights.h5')


In [6]:
ls

androidtest_model.h5    CNN.ipynb  dhrcdata/            Untitled.ipynb
androidtest_weights.h5  dev.py     new_cnn_may14.ipynb
